# Upload my predictions to local postgres db

In [1]:
# import what I need
from sqlalchemy import create_engine
from sqlalchemy_utils import database_exists, create_database
import psycopg2
import pandas as pd

In [2]:
# define folder locations, vars
cities = ['NEW+YORK', 'LOS+ANGELES']
folder = './produce_predictions/'

dbname = 'produce_predictions'
username = 'donald'


In [3]:
# engine
engine = create_engine('postgres://%s@localhost/%s'%(username,dbname))
print(engine.url)

postgres://donald@localhost/produce_predictions


In [4]:
## create a database (if it doesn't exist)
if not database_exists(engine.url):
    create_database(engine.url)
print(database_exists(engine.url))

True


In [5]:
# now read in each city's predictions
# delete current entries in database if they exist

for c in cities:
    predictions = pd.read_csv(folder+c+'_predictions.csv', index_col = 0)
    c = c.replace("+", "")
    c = c.lower()
    predictions = predictions.round(2)
    ## insert data into database from Python (proof of concept - this won't be useful for big data, of course)
    predictions.to_sql(c+'_predictions', engine, if_exists='replace')

In [6]:
dbname = 'produce_predictions'
user = 'donald'
con = psycopg2.connect(database = dbname, user = user)

# double check to make sure it actually worked
sql_query = """
          SELECT "Item", "Current Price", "3 Month Prediction" FROM "newyork_prophet_predictions";
           """
query_results=pd.read_sql_query(sql_query,con)
predictions = []
for i in range(0,query_results.shape[0]):
    predictions.append(dict(item=query_results.iloc[i]['Item'], currprice=query_results.iloc[i]['Current Price'], predictprice=query_results.iloc[i]['3 Month Prediction']))
print(predictions)

[{'item': 'APPLES', 'currprice': 28.05, 'predictprice': 24.01}, {'item': 'APRICOTS', 'currprice': 37.65, 'predictprice': 34.66}, {'item': 'ASPARAGUS', 'currprice': 26.9, 'predictprice': 16.66}, {'item': 'AVOCADOS', 'currprice': 33.71, 'predictprice': 34.66}, {'item': 'BANANAS', 'currprice': 16.53, 'predictprice': 16.24}, {'item': 'BEANS', 'currprice': 25.05, 'predictprice': 21.94}, {'item': 'BEETS', 'currprice': 13.07, 'predictprice': 7.3}, {'item': 'BLACKBERRIES', 'currprice': 18.4, 'predictprice': 16.77}, {'item': 'BLUEBERRIES', 'currprice': 31.23, 'predictprice': 27.25}, {'item': 'BROCCOLI', 'currprice': 29.02, 'predictprice': 15.77}, {'item': 'BRUSSELS+SPROUTS', 'currprice': 25.0, 'predictprice': 39.18}, {'item': 'CABBAGE', 'currprice': 12.96, 'predictprice': 7.7}, {'item': 'CANTALOUPS', 'currprice': 20.12, 'predictprice': 12.85}, {'item': 'CARROTS', 'currprice': 15.51, 'predictprice': 29.14}, {'item': 'CAULIFLOWER', 'currprice': 20.1, 'predictprice': 18.35}, {'item': 'CELERY', 'cu